# Qrious Technical Development Task

## Pokedex using PokeAPI:

In [1]:
import sys
import pandas as pd
import requests
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import io

In [2]:
response = requests.get('https://pokeapi.co/api/v2/pokemon/ditto')

# Checking if connection successful or not. 200 means successful. 
status = response.status_code

if status != 200:
    print("Error: Connection Unsuccessful")
    sys.exit()

response_json = response.json()

In [3]:
# Extract data from first 151 pokemon:

# Initialize emoty array to store pokemon names:
pokemon_names = []
response = requests.get(f'https://pokeapi.co/api/v2/pokemon/{2}')
name = response.json()['name']

# Extract all pokemon names and append to pokemon_names array:
for i in range(1, 152):
    response = requests.get(f'https://pokeapi.co/api/v2/pokemon/{i}')
    response_json = response.json()
    pokemon_names.append(response_json['name'])

In [4]:
# Extract all pokemon Number (id column in JSON) and append to pokemon_numbers array:
pokemon_numbers = []
for i in range(1, 152):
    response = requests.get(f'https://pokeapi.co/api/v2/pokemon/{i}')
    response_json = response.json()
    pokemon_numbers.append(response_json['id'])

In [5]:
# Extract all pokemon weights and append to pokemon_weights array:
pokemon_weights = []
for i in range(1, 152):
    response = requests.get(f'https://pokeapi.co/api/v2/pokemon/{i}')
    response_json = response.json()
    pokemon_weights.append(response_json['weight'])

In [6]:
# Extract all pokemon heights and append to pokemon_heights array:
pokemon_heights = []
for i in range(1, 152):
    response = requests.get(f'https://pokeapi.co/api/v2/pokemon/{i}')
    response_json = response.json()
    pokemon_heights.append(response_json['height'])

In [7]:
# Extract all pokemon base experiences and append to pokemon_base_experiences array:
pokemon_base_experiences = []
for i in range(1, 152):
    response = requests.get(f'https://pokeapi.co/api/v2/pokemon/{i}')
    response_json = response.json()
    pokemon_base_experiences.append(response_json['base_experience'])

In [8]:
# Extract all pokemon abilities and append to pokemon_abilities array:
pokemon_abilities = []
for i in range(1, 152):
    response = requests.get(f'https://pokeapi.co/api/v2/pokemon/{i}')
    response_json = response.json()
    pokemon_abilities.append(response_json['abilities'][0]['ability']['name'])

In [9]:
# create new feature of string w combined position name and number: 
pokemon_names_numbers = []

for i in range(0, 151):
    pokemon_names_numbers.append(pokemon_names[i] + " " + str(pokemon_numbers[i]))

In [10]:
# organise extracted data to a pandas dataframe:
pokemon_df = pd.DataFrame({
    'Number': pokemon_numbers,
    'Name': pokemon_names,
    'Weight': pokemon_weights,
    'Height': pokemon_heights,
    'Base Experience': pokemon_base_experiences,
    'Ability': pokemon_abilities,
    'NameNumber' : pokemon_names_numbers
})

# reorganise data so can access pokemon details by name:
pokemon_df_name = pokemon_df.set_index('NameNumber')

In [11]:
# function to filter list of pokemon using a string, which matches on pokemon name and number:
def filter_pokemon(pokemon_df, filter_string):
    return pokemon_df[pokemon_df['NameNumber'].str.contains(filter_string, case=False)]

In [12]:
def fetch_pokemon_data():
    pokemon_name = entry.get().lower()
    if not pokemon_name:
        messagebox.showerror("Error", "Please enter a Pokemon name.")
        return
    url = f'https://pokeapi.co/api/v2/pokemon/{pokemon_name}'
    response = requests.get(url)
    
    if response.status_code == 404:
        messagebox.showerror("Error", f"Pokemon '{pokemon_name}' not found.")
        return
    elif response.status_code != 200:
        messagebox.showerror("Error", "An error occurred.")
        return
    elif response.status_code == 200:
        data = response.json()
        display_pokemon_data(data)

def fetch_and_display_all_pokemon():
    url = 'https://pokeapi.co/api/v2/pokemon?limit=151'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        result_text.delete(1.0, tk.END) 
        for pokemon in data['results']:
            pokemon_name = pokemon['name']
            pokemon_url = pokemon['url']
            pokemon_response = requests.get(pokemon_url)
            if pokemon_response.status_code == 200:
                pokemon_data = pokemon_response.json()
                result_text.insert(tk.END, f"Name: {pokemon_name.capitalize()}\n")
                result_text.insert(tk.END, f"Number: {pokemon_data['id']}\n")
                result_text.insert(tk.END, f"Weight (lbs): {pokemon_data['weight']}\n")
                result_text.insert(tk.END, f"Height (ft): {pokemon_data['height']}\n")
                result_text.insert(tk.END, f"Base Experience: {pokemon_data['base_experience']}\n")
                result_text.insert(tk.END, "Abilities:\n")
                for ability in pokemon_data['abilities']:
                    result_text.insert(tk.END, f"- {ability['ability']['name']}\n")
                result_text.insert(tk.END, "Types: " + ", ".join([t['type']['name'] for t in pokemon_data['types']]) + "\n\n")
                result_text.insert(tk.END, "\n")
    else: 
        messagebox.showerror("Error", "Failed to fetch Pokemon list.")
            
def display_pokemon_data(data):
   
    result_text.delete(1.0, tk.END)
    result_text.insert(tk.END, f"Name: {data['name'].capitalize()}\n")
    result_text.insert(tk.END, f"Number: {data['id']}\n")
    result_text.insert(tk.END, f"Weight (lbs): {data['weight']}\n")
    result_text.insert(tk.END, f"Height (ft): {data['height']}\n")
    result_text.insert(tk.END, f"Base Experience: {data['base_experience']}\n")
    result_text.insert(tk.END, "Abilities:\n")
    for ability in data['abilities']:
        result_text.insert(tk.END, f"- {ability['ability']['name']}\n") 
    result_text.insert(tk.END, "Types: " + ", ".join([t['type']['name'] for t in data['types']]) + "\n")
    
    # Fetch and display the Pokemon's image:
    image_url = data['sprites']['front_default']
    if image_url: 
        image_response = requests.get(image_url)
        image_data = image_response.content
        image = Image.open(io.BytesIO(image_data))
        image = image.resize((100, 100), Image.ANTIALIAS)
        photo = ImageTk.PhotoImage(image)
        image_label.config(image=photo)
        image_label.image = photo
    else: 
        image_label.config(image='')
        
def sort_by_number_descending():
    url = 'https://pokeapi.co/api/v2/pokemon?limit=151'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        pokemon_list = []
        for pokemon in data['results']:
            pokemon_url = pokemon['url']
            pokemon_response = requests.get(pokemon_url)
            if pokemon_response.status_code == 200:
                pokemon_data = pokemon_response.json()
                pokemon_list.append(pokemon_data)
        
        # Sort the list by number (ID) in descending order
        sorted_pokemon_list = sorted(pokemon_list, key=lambda x: x['id'], reverse=True)
        
        result_text.delete(1.0, tk.END)
        for pokemon_data in sorted_pokemon_list:
            result_text.insert(tk.END, f"Name: {pokemon_data['name'].capitalize()}\n")
            result_text.insert(tk.END, f"Number: {pokemon_data['id']}\n")
            result_text.insert(tk.END, f"Weight (lbs): {pokemon_data['weight']}\n")
            result_text.insert(tk.END, f"Height (ft): {pokemon_data['height']}\n")
            result_text.insert(tk.END, f"Base Experience: {pokemon_data['base_experience']}\n")
            result_text.insert(tk.END, "Abilities:\n")
            for ability in pokemon_data['abilities']:
                result_text.insert(tk.END, f"- {ability['ability']['name']}\n") 
            result_text.insert(tk.END, "Types: " + ", ".join([t['type']['name'] for t in pokemon_data['types']]) + "\n")
            result_text.insert(tk.END, "\n")
    else:
        messagebox.showerror("Error", "Failed to fetch Pokémon list.")
        
def sort_by_number_ascending():
    url = 'https://pokeapi.co/api/v2/pokemon?limit=151'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        pokemon_list = []
        for pokemon in data['results']:
            pokemon_url = pokemon['url']
            pokemon_response = requests.get(pokemon_url)
            if pokemon_response.status_code == 200:
                pokemon_data = pokemon_response.json()
                pokemon_list.append(pokemon_data)
        
        # Sort the list by number (ID) in ascending order
        sorted_pokemon_list = sorted(pokemon_list, key=lambda x: x['id'])
        
        result_text.delete(1.0, tk.END)
        for pokemon_data in sorted_pokemon_list:
            result_text.insert(tk.END, f"Name: {pokemon_data['name'].capitalize()}\n")
            result_text.insert(tk.END, f"Number: {pokemon_data['id']}\n")
            result_text.insert(tk.END, f"Weight (lbs): {pokemon_data['weight']}\n")
            result_text.insert(tk.END, f"Height (ft): {pokemon_data['height']}\n")
            result_text.insert(tk.END, f"Base Experience: {pokemon_data['base_experience']}\n")
            result_text.insert(tk.END, "Abilities:\n")
            for ability in pokemon_data['abilities']:
                result_text.insert(tk.END, f"- {ability['ability']['name']}\n") 
            result_text.insert(tk.END, "Types: " + ", ".join([t['type']['name'] for t in pokemon_data['types']]) + "\n")
            result_text.insert(tk.END, "\n")
    else:
        messagebox.showerror("Error", "Failed to fetch Pokémon list.")

def sort_by_alphabetical_ascending():
    url = 'https://pokeapi.co/api/v2/pokemon?limit=151'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        pokemon_list = []
        for pokemon in data['results']:
            pokemon_url = pokemon['url']
            pokemon_response = requests.get(pokemon_url)
            if pokemon_response.status_code == 200:
                pokemon_data = pokemon_response.json()
                pokemon_list.append(pokemon_data)
        
        # Sort the list by name in ascending order (A-Z)
        sorted_pokemon_list = sorted(pokemon_list, key=lambda x: x['name'])
        
        result_text.delete(1.0, tk.END)
        for pokemon_data in sorted_pokemon_list:
            result_text.insert(tk.END, f"Name: {pokemon_data['name'].capitalize()}\n")
            result_text.insert(tk.END, f"Number: {pokemon_data['id']}\n")
            result_text.insert(tk.END, f"Weight (lbs): {pokemon_data['weight']}\n")
            result_text.insert(tk.END, f"Height (ft): {pokemon_data['height']}\n")
            result_text.insert(tk.END, f"Base Experience: {pokemon_data['base_experience']}\n")
            result_text.insert(tk.END, "Abilities:\n")
            for ability in pokemon_data['abilities']:
                result_text.insert(tk.END, f"- {ability['ability']['name']}\n") 
            result_text.insert(tk.END, "Types: " + ", ".join([t['type']['name'] for t in pokemon_data['types']]) + "\n")
            result_text.insert(tk.END, "\n")
    else:
        messagebox.showerror("Error", "Failed to fetch Pokémon list.")

def sort_by_alphabetical_descending():
    url = 'https://pokeapi.co/api/v2/pokemon?limit=151'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        pokemon_list = []
        for pokemon in data['results']:
            pokemon_url = pokemon['url']
            pokemon_response = requests.get(pokemon_url)
            if pokemon_response.status_code == 200:
                pokemon_data = pokemon_response.json()
                pokemon_list.append(pokemon_data)
        
        # Sort the list by name in descending order (Z-A)
        sorted_pokemon_list = sorted(pokemon_list, key=lambda x: x['name'], reverse=True)
        
        result_text.delete(1.0, tk.END)
        for pokemon_data in sorted_pokemon_list:
            result_text.insert(tk.END, f"Name: {pokemon_data['name'].capitalize()}\n")
            result_text.insert(tk.END, f"Number: {pokemon_data['id']}\n")
            result_text.insert(tk.END, f"Weight (lbs): {pokemon_data['weight']}\n")
            result_text.insert(tk.END, f"Height (ft): {pokemon_data['height']}\n")
            result_text.insert(tk.END, f"Base Experience: {pokemon_data['base_experience']}\n")
            result_text.insert(tk.END, "Abilities:\n")
            for ability in pokemon_data['abilities']:
                result_text.insert(tk.END, f"- {ability['ability']['name']}\n") 
            result_text.insert(tk.END, "Types: " + ", ".join([t['type']['name'] for t in pokemon_data['types']]) + "\n")
            result_text.insert(tk.END, "\n")
    else:
        messagebox.showerror("Error", "Failed to fetch Pokémon list.")

# create a main application window: 
root = tk.Tk()
root.title("Pokedex")
# Set the window size to full screen:
root.geometry("8000x6000")

# Create and place the widgets:
tk.Label(root, text="Enter Pokemon Name:").pack(pady=5) 
entry = tk.Entry(root)
entry.pack(pady=5)

tk.Button(root, text="Fetch Pokemon's Data", command=fetch_pokemon_data).pack(pady=5)
tk.Button(root, text="Sort Pokemon in Descending Order", command=sort_by_number_descending).pack(pady=5)
tk.Button(root, text="Sort Pokemon in Ascending Order", command=sort_by_number_ascending).pack(pady=5)
tk.Button(root, text="Sort Pokemon by name (A-Z)", command=sort_by_alphabetical_ascending).pack(pady=5)
tk.Button(root, text="Sort Pokemon by name (Z-A)", command=sort_by_alphabetical_descending).pack(pady=5)
result_text = tk.Text(root, height=45, width=150)
result_text.pack(pady=5)

# Add a label to display the Pokemon image:
image_label = tk.Label(root)
image_label.pack(pady=5)

# Fetch and display all Pokemon data:
fetch_and_display_all_pokemon()

# Start the main event loop:
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\yasmi\anaconda3\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\yasmi\AppData\Local\Temp\ipykernel_20676\2647304779.py", line 17, in fetch_pokemon_data
    display_pokemon_data(data)
  File "C:\Users\yasmi\AppData\Local\Temp\ipykernel_20676\2647304779.py", line 63, in display_pokemon_data
    image = image.resize((100, 100), Image.ANTIALIAS)
                                     ^^^^^^^^^^^^^^^
AttributeError: module 'PIL.Image' has no attribute 'ANTIALIAS'
